In [1]:
import torch
import torch.nn.functional as F

In [2]:
def softmax(X: torch.Tensor):
    m = X.flatten().clone()
    d = torch.ones_like(m)
    for i in range(1, m.shape[0]):
        mi, mj = m[i-1], m[i]
        di, dj = d[i-1], d[i]
        new_max = max(mi, mj)
        new_sum = di * torch.exp(mi - new_max) + dj * torch.exp(mj - new_max)
        # print(torch.exp(mi - t[0][i]).item(), torch.exp(mj - t[0][i]).item())
        m[i] = new_max
        d[i] = new_sum
        
        


    mv, dv = m[-1], d[-1]
    print(f'dv is {dv}')
    return (X-mv).exp() / dv.item()


A = torch.tensor([1, 30, 0, 40, 5, 3, 4, 12], dtype=torch.float32, device='cuda')
softmax(A), F.softmax(A)

dv is 1.0000454187393188


/tmp/ipykernel_15787/3645033109.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  softmax(A), F.softmax(A)


(tensor([1.1548e-17, 4.5398e-05, 4.2482e-18, 9.9995e-01, 6.3048e-16, 8.5327e-17,
         2.3194e-16, 6.9141e-13], device='cuda:0'),
 tensor([1.1548e-17, 4.5398e-05, 4.2482e-18, 9.9995e-01, 6.3048e-16, 8.5327e-17,
         2.3194e-16, 6.9141e-13], device='cuda:0'))

In [3]:
# d is a running sum of exponentiated values

m = 2 
# d = e^m
d = torch.e ** m
print(d)

# d = e^m -> e^n







m = 5

7.3890560989306495


In [4]:
from torch.utils.cpp_extension import load
module = load(
    name='m',
    sources=['main.cpp', 'softmax.cu'],
    verbose=True,
    build_directory='cuda-build'

)

No ROCm runtime is found, using ROCM_HOME='/opt/rocm-6.2.0'
Detected CUDA files, patching ldflags
Emitting ninja build file cuda-build/build.ninja...
/home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/utils/cpp_extension.py:1967: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module m...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/3] c++ -MMD -MF main.o.d -DTORCH_EXTENSION_NAME=m -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/TH -isystem /home/seb/CUDA/cudaenv/lib/python3.12/site-packages/torch/include/THC -isystem /usr/local/cuda-12.3/include -isystem /home/seb/miniconda3/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -fPIC -std=c++17 -c /home/seb/CUDA/flash-attention/online_softmax/main.cpp -o main.o 
[2/3] /usr/local/cuda-12.3/bin/nvcc --generate-dependencies-with-compile --dependency-output softmax.cuda.o.d -DTORCH_EXTENSION_NAME=m -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /

Loading extension module m...


In [14]:
B = torch.rand(29, device='cuda', dtype=torch.float32)
# A = torch.tensor([1, 0, 4, 5], dtype=torch.float32, device='cuda')
module.softmax(B), softmax(B)

dv is 19.02651023864746


(tensor([0.0222, 0.0452, 0.0366, 0.0484, 0.7041, 0.7839, 0.3832, 0.4354, 0.7191,
         0.4597, 0.3467, 0.4764, 0.1350, 0.8765, 0.7420, 0.2697, 0.6575, 0.1705,
         0.2379, 0.1659, 0.3326, 0.9253, 0.7323, 0.8366, 0.2354, 0.7464, 0.3698,
         0.7602, 0.9653], device='cuda:0'),
 tensor([0.0222, 0.0452, 0.0366, 0.0484, 0.0254, 0.0359, 0.0526, 0.0256, 0.0341,
         0.0359, 0.0280, 0.0328, 0.0300, 0.0510, 0.0432, 0.0400, 0.0426, 0.0298,
         0.0331, 0.0241, 0.0379, 0.0376, 0.0245, 0.0462, 0.0211, 0.0309, 0.0248,
         0.0279, 0.0326], device='cuda:0'))

In [6]:
# Stride is 2
# Old value = 1.000000, New value = 1.606531
# Stride is 1
# cuda: mv is 1.000000, dv is 2.342289
# mv is 1.0, dv is 4.0
16*3

48